In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv('../data/Extracted_Attributes.csv')
data['Date']= pd.to_datetime(data['Date'])

In [3]:
data.Currency.value_counts(dropna=False)

$      2528
NaN     222
eur     180
usd     123
e        49
€        33
dem      26
frf      14
c         8
jpy       5
¥         5
uss       3
sdr       2
s         2
£         1
egp       1
us        1
y         1
gbp       1
Name: Currency, dtype: int64

Convert the loan amounts in US dollars.

In [4]:
#How many without loan paragraph
data.Loan_paragraph.isnull().sum()

78

In [5]:
#Lets look at a couple of random examples per currency to decide what they are likely to be
Sample_DF=data[['Currency','Country_code_pdf','Loan_paragraph']].groupby('Currency').apply(lambda x: x.sample(1)).reset_index(drop=True)
with pd.option_context('display.max_colwidth', -1):
   display(Sample_DF)

,Currency,Country_code_pdf,Loan_paragraph
0,$,PH,102 unfess the context requires otherwise the capitalized terms used in this agreement have the meanings ascribed to them in the general conditions or in the appendix to this agreement article ii loan 201 the bank agrees to lend to the borrower on the terms and conditions set forth or referred to in this agreement the amount of ninetynine million three hundred twelve thousand nine hundred and ten dollars ($99312910) as such amount may be converted from time to time through a currency conversion in accordance with the provisions of section 208 of this agreement (loan) to assist in financing the project described in schedule 1 to this agreement (project)
1,c,PL,102 unless the context requires otherwise the capitalized terms used in this agreement have the meanings ascribed to them in the general conditions or in the appendix to this agreement article ii loan 201 the bank agrees to lend to the borrower on the terms and conditions set forth or referred to in this agreement the amount of seven hundred million euro (c700000000) as such amount may be converted from time to time through a currency conversion in accordance with the provisions of section 207 of this agreement (loan)
2,dem,MK,article ii the loan section 201 the bank agrees to lend to the borrower on the terms and conditions set forth or referred to in the loan agreement an amount equal to thirteen million five hundred thousand deutsche mark (dem 13500000)
3,e,BA,102 unless the context requires otherwise the capitalized terms used in this agreement have the meanings ascribed to them in the general conditions or in the appendix to this agreement article ii loan 201 the bank agrees to lend to the borrower the amount of fiftyone million three hundred thousand euro (e51300000) as such amount may be converted from time to time through a currency conversion (loan) to assist in financing the project described in schedule i to this agreement (project)
4,egp,EGT,constitute an integral part of this agreement 102 unless the context requires otherwise the capitalized terms used in the loan agreement have the meanings ascribed to them in the general conditions or in the appendix to this agreement article ii loan 201 the bank agrees to lend to the borrower on the terms and conditions set forth or referred to in this agreement the amount of two hundred fourteen million two hundred thousand egyptian pounds (egp214200000) (loan) to assist in financing the project described in schedule 1 to this agreement (project)
5,eur,YF,102 unless the context requires otherwise the capitalized terms used in this agreement have the meanings ascribed to them in the general conditions or in the appendix to this agreement article ii loan 201 the bank agrees to lend to the borrower on the terms and conditions set forth or referred to in this agreement the amount of sixtysix million one hundred thousand euros (eur 66100000) as such amount may be converted from time to time through a currency conversion in accordance with the provisions of section 208 of this agreement (loan)
6,frf,MOR,maintained pursuant to annex 4 of the convention article ii the loan section 201 the bank agrees to lend to the borrower on the terms and conditions set forth or referred to in the loan agreement an amount equal to forty million nine hundred thousand french francs (frf40900000)
7,gbp,MU,in this agreement have the meanings ascribed to them in the general conditions or in the appendix to this agreement article ii loan 201 the bank agrees to lend to the borrower on the terms and conditions set forth or referred to in this agreement the amount of nine million six hundred thousand pounds (gbp 9600000) as such amount may be converted from time to time through a currency conversion in accordance with the provisions of section 207 of this agreement (loan)
8,jpy,ID,10l the general conditions (as defined in the appendix to this agreement) apply to and form part of this agreement 102 unless

In [6]:
#From the above we recognize the following currencies
currency_names=['dollar','euro','yen','deutsche mark','egyptian pound','sterling','franc','drawing right']

#Map the currencies we are sure are unique
map_currencies={'$':'dollar', 'dem':'deutsche mark', 'egp':'egyptian pound',
                'eur':'euro',  'frf':'franc', 'gbp':'sterling',
                'jpy':'yen','sdr':'drawing right', 'us':'dollar',
                'usd':'dollar','£':'sterling','¥':'yen','€':'euro', 'e':'euro'}
Map_Currencies_DF=pd.DataFrame({'Currency':map_currencies.keys(), 'Currency_name':map_currencies.values()})

In [7]:
Map_Currencies_DF

,Currency,Currency_name
0,$,dollar
1,dem,deutsche mark
2,egp,egyptian pound
3,eur,euro
4,frf,franc
5,gbp,sterling
6,jpy,yen
7,sdr,drawing right
8,us,dollar
9,usd,dollar


In [8]:
data=data.merge(Map_Currencies_DF,how='outer',on='Currency')

In [9]:
#Function to extract loan amount using currency name instead of symbol.
def get_loan_amount_currency(paragraph,currency_names):
    paragraph=str(paragraph)
    #Extract dollar amount
    #Extract the values using Regular Expressions
    currency_names_reg='|'.join(currency_names)
    regexp_amount='('+currency_names_reg+').*?([ \d]{6,})'

    value=re.search(regexp_amount,paragraph)
    if value!=None:
        currency_name=value.group(1)
        amount=value.group(2)
        amount=amount.replace(' ','')
        amount=int(amount)
        #Make double sure that the amount makes sense. 
        if amount>100000:
            return amount,currency_name

    #If we couldnt find an amount, return None
    return None,None

In [10]:
#Try to extract loan amount again
for ind in data[data.Currency_name.isnull()].index:
    amount,currency_name=get_loan_amount_currency(data.Loan_paragraph[ind],currency_names)
    data['Currency_name'][ind]=currency_name
    data['Amount_loan'][ind]=amount

<ipython-input-10-bdb3b8e2ab2e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Currency_name'][ind]=currency_name
<ipython-input-10-bdb3b8e2ab2e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Amount_loan'][ind]=amount


In [11]:
data.Currency_name.isnull().sum()

88

In [12]:
data.groupby(['Currency','Currency_name'],dropna=False).count()[['filename']]

,,filename
Currency,Currency_name,
$,dollar,2528
c,euro,8
dem,deutsche mark,26
e,euro,49
egp,egyptian pound,1
eur,euro,180
frf,franc,14
gbp,sterling,1
jpy,yen,5


In [13]:
exchangerate = pd.read_csv('../exchangerates.csv')
exchangerate=exchangerate.rename({'TIME':'Year', 'LOCATION':'Currency_name'},axis=1)
exchangerate=exchangerate[['Year','Currency_name','Value']]

In [14]:
#We now convert currency names into the names that the exchangerates.csv manage
Table_Currency={'dollar':'USD',
               'euro':'EU27_2020',
               'yen':'JPN',
               'deutsche mark':'DEU',
               'egyptian pound':'EGP',
               'sterling':'GBR',
               'franc':'FRA',
               'drawing right':'XDR'}
data=data.replace({'Currency_name':Table_Currency})
data['Year']=pd.Series([date.year for date in data.Date])

In [15]:
#Do the join
data=data.merge(exchangerate, how='left', on=['Year','Currency_name'])

#Value for dollars should be 1
data['Value'][data.Currency_name=='USD']=1

<ipython-input-15-9d736deeb457>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Value'][data.Currency_name=='USD']=1


In [16]:
data['Amount_in_usd']=data['Amount_loan']/data['Value']

In [17]:
data.Amount_in_usd.isnull().sum()

91

In [18]:
Export_DF=data.drop(['Loan_paragraph','Currency'],axis=1)

In [19]:
Export_DF.to_csv('../data/Extracted_Attributes_LoanUSD.csv',index=False)